# Import TensorFlow 2.x.

In [ ]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

print(tf.__version__)

# Import TensorFlow datasets.
*   MNIST dataset

In [ ]:
import tensorflow_datasets as tfds

# Import TensorFlow addons.
* Triplet semi-hard loss

In [ ]:
import tensorflow_addons as tfa

# Load MNIST dataset.

### Load MNIST dataset.
* train split
* test split

In [ ]:
train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

### Normalize dataset images.

In [ ]:
def _normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.
    return (image, label)

### Create dataset batches.

In [ ]:
buffer_size = 1024
batch_size = 32

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
train_dataset = train_dataset.map(_normalize_image)

test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(_normalize_image)

# Create the model.
* No activation (or default linear activation) on last layer
* L2 normalized embeddings.

In [ ]:
model = models.Sequential([
    layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),

    layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),

    layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),
    
    layers.Flatten(),
    layers.Dense(256, activation=None),                      # No activation on final dense layer
    layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalized embeddings
])

### Show model summary.

In [ ]:
model.summary()

# Train the model.

### Compile the model.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tfa.losses.TripletSemiHardLoss())

### Train the model.

In [ ]:
history = model.fit(train_dataset, epochs=10)

# Evaluate the model.
* Create the embeddings for the test dataset.



In [ ]:
embeddings = model.predict(test_dataset)

# Save the embeddings for visualization in the embedding projector.

In [ ]:
import io

np.savetxt("embeddings-vecs.tsv", embeddings, delimiter='\t')

meta_file = io.open('embeddings-meta.tsv', 'w', encoding='utf-8')
for image, labels in tfds.as_numpy(test_dataset):
    [meta_file.write(str(label) + "\n") for label in labels]
meta_file.close()

# Visualize using Embedding Projector.

Generated embedding vector and metadata files can be loaded and visualized using Embedding Projector available [here](https://projector.tensorflow.org).